# Introduction to `fastplotlib`

## `fastplotlib` API

### 1. Graphics - objects that are drawn
- `Image`, `Line`, `Scatter`, `Heatmap`
- Collections - `LineCollection`, `LineStack` (ex: neural timeseries data)
- Interactions

### 2. Layouts
- `Plot` - a single plot area 
- `GridPlot` - a grid of `Subplots`

### 3. Widgets - high level widgets to make repetitive UIs easier
- `ImageWidget`- n-dimensional widget for `Image` data
    - Sliders, support window functions, `GridPlot`, etc.
    
    

# Running demos on an ancient Quadro P500 laptop GPU

## nearly 6 years old

Benchmark - 1,555

Nothing compared to midrange GPU, RTX 3060 Ti: 20,633

https://www.videocardbenchmark.net/gpu.php?gpu=Quadro+P500&id=3911

This notebook goes through basic components of the `fastplotlib` API including images, image updates, and `ImageWidget`.

**The example images are from imageio so you will need to install it for this example notebook. But imageio is not required to use fasptlotlib**

In [1]:
import imageio.v3 as iio

# likewise there is an image which we will load with tifffile
import tifffile

In [2]:
import fastplotlib as fpl
from ipywidgets import VBox, HBox, IntSlider
import numpy as np
from sidecar import Sidecar

### Simple image

In [3]:
# create a `Plot` instance
plot = fpl.Plot(size=(600, 500))

# get a grayscale image
data = iio.imread("imageio:camera.png")

# plot the image data
image_graphic = plot.add_image(data=data, name="sample-image")

# display plot in sidecar
sc = Sidecar(title="sample image", layout={'width': '800px'})

with sc:
    # show the plot
    display(plot.show())

RFBOutputContext()

MESA-INTEL: warning: Performance support disabled, consider sysctl dev.i915.perf_stream_paranoid=0

Disabling presentation on 'Intel(R) UHD Graphics 620 (KBL GT2)' (id 0x55a616dc6870) because of NV Optimus (on Linux)


**Use the handle on the bottom right corner of the _canvas_ to resize it. You can also pan and zoom using your mouse!**

By default the origin is on the bottom left, you can click the flip button to flip the y-axis, or use `plot.camera.world.scale_y *= -1`

Changing graphic **"features"**

In [4]:
image_graphic.cmap = "viridis"

### Slicing data

**Most features, such as `data`, support slicing!**

Our image data is of shape [n_rows, n_columns]

In [5]:
image_graphic.data[::15, :] = 1
image_graphic.data[:, ::15] = 1

**Fancy indexing**

In [6]:
image_graphic.data[data > 175] = 255

**Adjust vmin vmax**

In [7]:
image_graphic.cmap.vmin = 50
image_graphic.cmap.vmax = 150

In [8]:
image_graphic.cmap.reset_vmin_vmax()

In [9]:
# close plot and sidecar
plot.close()
sc.close()

**Note: if you have a good enough gpu, you do not need to close the plots. We are doing this here for gpu conservation.**

### Image updates

This examples show how you can define animation functions that run on every render cycle.

In [10]:
# create another `Plot` instance
plot_v = fpl.Plot(size=(600, 500))

plot.canvas.max_buffered_frames = 1

# make some random data again
data = np.random.rand(512, 512)

# plot the data
plot_v.add_image(data=data, name="random-image")

# a function to update the image_graphic
# a plot will pass its plot instance to the animation function as an arugment
def update_data(plot_instance):
    new_data = np.random.rand(512, 512)
    plot_instance["random-image"].data = new_data

#add this as an animation function
plot_v.add_animations(update_data)

# sidecar to display plot
sc = Sidecar(title="image updates")

with sc:
    # show the plot
    display(plot_v.show())

RFBOutputContext()

/home/kushal/repos/fastplotlib/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


### We can share controllers across plots

This example creates a new plot, but it synchronizes the pan-zoom controller

In [11]:
plot_sync = fpl.Plot(controller=plot_v.controller, size=(600, 500))

data = np.random.rand(512, 512)

image_graphic_instance = plot_sync.add_image(data=data, cmap="viridis")

# you will need to define a new animation function for this graphic
def update_data_2():
    new_data = np.random.rand(512, 512)
    # alternatively, you can use the stored reference to the graphic as well instead of indexing the Plot
    image_graphic_instance.data = new_data

plot_sync.add_animations(update_data_2)

plot_sync.show()

RFBOutputContext()

#### Keeping a reference to the Graphic instance, as shown above `image_graphic_instance`, is useful if you're creating something where you need flexibility in the naming of the graphics

In [12]:
sc.close()

### You can also use `ipywidgets.VBox` and `HBox` to stack plots.

In [13]:
VBox([plot_v.show(), plot_sync.show()])

In [14]:
# close plots
plot_v.close()
plot_sync.close()

# Sliders to scroll through image data

We often already have large image arrays (whether in RAM or through lazy loading), and want to view 2D frames across one or more dimensions. There is an `ImageWidget` that should really be used for this, but this example just shows how you can use `ipywidgets` to change data or any **`GraphicFeature`**

## Plot and scroll through the first dimension with a slider

In [15]:
# use imagio cockatoo video
movie = iio.imread('imageio:cockatoo.mp4', index=None)

In [16]:
plot_movie = fpl.Plot()

# plot the first frame to initialize
movie_graphic = plot_movie.add_image(movie[0], vmin=0, vmax=movie.max(), cmap="gray")

# make a slider
slider = IntSlider(min=0, max=movie.shape[0] - 1, step=1, value=0)

# function to update movie_graphic
def update_movie(change):    
    index = change["new"]
    movie_graphic.data = movie[index]
    
slider.observe(update_movie, "value")
    
# Use an ipywidgets VBox to show the plot and slider
VBox([plot_movie.show(), slider])

RFBOutputContext()

#### Note that the use of globals in the `update_movie()` here can get messy, this is not recommended and  you should create a class to properly handle combining widgets like this. _However_ if you want slider widgets for imaging data the recommended way to do this is by using the `ImageWidget`.

In [17]:
plot_movie.close()

## `ImageWidget` using zebrafish data

In [18]:
# load in zebrafish data
movie = tifffile.memmap('./data/zfish_vol.tiff')

In [19]:
movie.shape

(1320, 30, 512, 512)

This is `[t, z, x, y]`

In [20]:
iw = fpl.ImageWidget(
    data=movie,
    vmin_vmax_sliders=True,
    dims_order="tzxy",
    slider_dims=["t", "z"],
    grid_plot_kwargs={"size": (600, 500)},
    cmap="gnuplot2"
)

RFBOutputContext()

In [21]:
sc = Sidecar(title="zebra fish")

with sc:
    display(iw.show())

### Play with setting different window functions

These can also be given as kwargs to `ImageWidget` during instantiation

In [22]:
# must be in the form of {dim: (func, window_size)}
iw.window_funcs = {"t": (np.mean, 3)}

In [23]:
iw.gridplot.close()
sc.close()

## View each plane in a separate subplot

In [25]:
# get a list of txy arrays for each plane
planes = [movie[:, i] for i in range(movie.shape[1])]

iw = fpl.ImageWidget(
    data=planes,
    vmin_vmax_sliders=False,
    grid_plot_kwargs={"size": (600, 500), "controllers": "sync"},
    cmap="gnuplot2"
)


sc = Sidecar(title="zfish planes")

with sc:
    display(iw.show())

RFBOutputContext()

In [26]:
iw.gridplot.close()
sc.close()